# Assembly of system with multiple domains, variables and numerics

This tutorial has the dual purpose of illustrating parameter assigment in PorePy, and also showing  how to set up problems in (mixed-dimensional) geometries. Starting from standard discretizations of problems with a single primary variable, the tutorial will gradually be expanded to cover
* Equations with multiple terms (e.g. advection-diffusion equations)
* Equations where a term is treated with different discretization methods on different domains
* Equations with multiple variables
* Equations where some variables are only present in some subdomains

Not all of these items are currently available, we are working on it.


In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp

In [2]:
def make_grid():
    # Define a grid for a toy problem
    f1 = np.array([[0, 1], [0.5, 0.5]])

    gb = pp.meshing.cart_grid([f1], [2, 2])
    gb.compute_geometry()
    gb.assign_node_ordering()
    return gb

In [3]:
def assign_data(gb, keyword_param_storage, keyword_param_storage_2=None):
    # Method to assign data.
    for g, d in gb:
        # We will mainly use default values for parameters, while overriding some of the 
        # values. Sets of default parameters are available for flow, transport and 
        # mechanics (elasticity). For example, initialize_default_data initializes a 
        # second order permeability tensor in the flow data, and a fourth order stiffness
        # tensor in the mechanics data. For more details, and definitions of what the defaults
        # are, see the modules pp/params/data.py and pp/params/parameter_dictionaries.py 

        # This keyword is used to define which set of default parameters to pick
        # Replace with 'transport' or 'mechanics' if needed
        default_parameter_keyword = 'flow'  

        # Assign a non-default permeability, for illustrative purposes
        if g.dim == 2:
            kxx = 10 * np.ones(g.num_cells)
        else:
            kxx = 0.1 * np.ones(g.num_cells)

        perm = pp.SecondOrderTensor(gb.dim_max(), kxx)

        # Create a dictionary to override the default parameters
        specified_parameters = {'second_order_tensor': perm}

        # The parameters are stored in a class pp.Parameters. 
        # This class is again stored in the data dictionary on each node and edge in 
        # the GridBucket, that is, in the variable d in this loop. The Paramater
        # object can be accessed by d[pp.PARAMETERS]
        # To allow storage of parameters for several problems simultaneously (say, we
        # want to solve a combined flow and transport problem), the Parameter class
        # uses keywords to identify sets of parameters. This keyword must also be provided
        # to the discretization method.
        # When the parameter class is initialized with default values, the default behavior
        # is to identify the parameters by the standard keyword (above
        # `default_parameter_keyword`). 
        # While this is usually good practice, we here override this behavior for illustrative
        # purposes, using the "keyword_param_storage"

        # Define the 
        pp.initialize_default_data(g, d, default_parameter_keyword, specified_parameters,
                                   keyword_param_storage)

        # Internally to the Parameter class, the parameters are stored as dictionaries.
        # To illustrate how to access specific sets of parameters, print the keywords
        # for one of the grids
        if g.dim == 2:
            print('The assigned parameters for the ')
            print(d[pp.PARAMETERS][keyword_param_storage].keys())
           
        # For one example below, we will need two different parameter sets.
        # Define a second set, with default values only.
        if keyword_param_storage_2:
            pp.initialize_default_data(g, d, default_parameter_keyword, keyword = keyword_param_storage_2)
        

    for e, d in gb.edges():
        # On edges in the GridBucket, there is currently no methods for default initialization.

        data = {"normal_diffusivity": 1e1}
        # Add parameters. We again use keywords to identify sets of parameters.
        if two_parameter_sets:
            d[pp.PARAMETERS] = pp.Parameters(keywords=['flow_param_edge', 'second_flow_param_edge'],
                                             dictionaries=[data, data])
        else:
            d[pp.PARAMETERS] = pp.Parameters(keywords=['flow_param_edge'], dictionaries=[data])
    
    return gb

In [32]:
# The practical way of setting up a problem with a single variable is described here.
# For explanations, and hints on how to consider a more general setting,
# see the expanded assignment below.

# Define a grid
gb = make_grid()

# Assign parameters, tagging them with a keyword. This keyword ensures that the
# discretizer (here tpfa, defined below) uses the right set of parameters. 
parameter_keyword = 'flow_param'
gb = assign_data(gb, parameter_keyword)


# Define the pressure variable with the same keyword on all grids
grid_variable = 'pressure'
# Variable name for the flux between grids, that is, the primary variable
# on the edges in the GridBucket.
mortar_variable = 'mortar_flux'

# Identifier of the discretization operator on each grid
operator_keyword = 'diffusion'
# Identifier of the discretization operator between grids
coupling_operator_keyword = 'coupling_operator'

# Use a two-point flux approximation on all grids.
# Note the keyword here: It must be the same as used when assigning the 
# parameters.
tpfa = pp.Tpfa(parameter_keyword)

# Between the grids we use a Robin type coupling (resistance to flow over a fracture).
# Again, the keyword must be the same as used to assign data to the edge
# The edge discretization also needs access to the corresponding discretizations 
# on the neighboring nodes
edge_discretization = pp.RobinCoupling('flow_param_edge', tpfa, tpfa)

# Loop over the nodes in the GridBucket, define primary variables and discretization schemes
for g, d in gb:
    # Assign primary variables on this grid. It has one degree of freedom per cell.
    d[pp.PRIMARY_VARIABLES] = {grid_variable: {"cells": 1, "faces": 0}}
    # Assign discretization operator for the variable.
    # If the discretization is composed of several terms, they can be assigned
    # by multiple entries in the inner dictionary, e.g.
    #  {operator_keyword_1: method_1, operator_keyword_2: method_2, ...}
    d[pp.DISCRETIZATION] = {grid_variable: {operator_keyword: tpfa}}
    
# Loop over the edges in the GridBucket, define primary variables and discretizations
for e, d in gb.edges():
    g1, g2 = gb.nodes_of_edge(e)
    # The mortar variable has one degree of freedom per cell in the mortar grid
    d[pp.PRIMARY_VARIABLES] = {mortar_variable: {"cells": 1}}
    
    # The coupling discretization links an edge discretization with variables
    # and discretization operators on each neighboring grid
    d[pp.COUPLING_DISCRETIZATION] = {
        coupling_operator_keyword: {
            g1: (grid_variable, operator_keyword),
            g2: (grid_variable, operator_keyword),
            e: (mortar_variable, edge_discretization),
        }
    }
    d[pp.DISCRETIZATION_MATRICES] = {'flow_param_edge': {}}
    
# The task of assembling the linear system is left to a dedicated object:
assembler = pp.Assembler()

# Discretization and assembly of the global linear system can in this case
# be carried out by a single function call.
# Note that for some problems, notably poro-elasticity, this is not possible,
# then discretization must be carried out first.

# Assemble the linear system, using the information stored in the GridBucket
A, b, block_dof, full_dof = assembler.assemble_matrix_rhs(gb)

# Here, A is the global linear system, and b is the corresponding right hand side.
# We can thus obtain the pressure solution by
pressure = sps.linalg.spsolve(A, b)
# The parameters assigned above will not yield a well-posed problem, 
# thus the solve will likely produce a warning about the matrix being singular.
# This can be ignored.

# The ordering of the unknowns in the global linear system will vary depending 
# on how the components in the GridBucket are traversed.
# Untangling the ordering is a two-stage process:
# 1. The system is ordered as a block system, with one block per combination of 
# primary variable and grid or edge (between grids). This information is stored
# in the field block_dof.
# 2. For each block, the local degrees of freedom can be obtained from the field
# full_dof

# To get the block number of a specific primary variable, we need the identifier
# of the relevant component in the GridBucket (either the grid, or the edge between
# grids), and the variable name.
# Getting the grids is easy, there is one in each dimension
g_2d = gb.grids_of_dimension(2)[0]
g_1d = gb.grids_of_dimension(1)[0]

# Formally loop over the edges, there is a single one
for e, _ in gb.edges():
    continue

# Now, the block ordering is obtained, for the 2d grid as 
block_2d = block_dof[(g_2d, grid_variable)]

# full_dof contains the number of dofs per block. To get a global ordering, use
global_dof = np.r_[0, np.cumsum(full_dof)]

# Get 2d dofs
global_dof_2d = np.arange(global_dof[block_2d], global_dof[block_2d+1])
# Print the relevant part of the system matrix
print(A.toarray()[global_dof_2d, :][:, global_dof_2d])



The assigned parameters for the 
dict_keys(['aperture', 'porosity', 'source', 'mass_weight', 'second_order_tensor', 'bc', 'bc_values', 'fluid_compressibility', 'time_step'])
[[ 20. -10. -10.   0.]
 [-10.  20.   0. -10.]
 [-10.   0.  20. -10.]
 [  0. -10. -10.  20.]]


In [ ]:
# Next, define primary variables and assign discretization methods on 
# using different names and objects on each component of the GridBucket.
# The resulting system will be somewhat arbitrary, in that it may not
# reflect any standard physics, but it should better illustrate what is
# needed for a multi-physics problem.

# Define a grid
gb = make_grid()
parameter_keyword = 'second_flow_param'
gb = assign_data(gb, parameter_keyword, parameter_keyword_2)


# Primary variables must be defined on each component of the GridBucket.
# Define the pressure variable with the same keyword on all grids
grid_1_variable_1 = 'pressure'
grid_1_variable_2 = 'foo'

grid_2_variable = 'flux_pressure'

# Variable name for the flux between grids, that is, the primary variable
# on the edges in the GridBucket.
mortar_variable = 'mortar_flux'

# Identifier of the discretization operator on each grid
operator_keyword = 'diffusion'
# Identifier of the discretization operator between grids
coupling_operator_keyword = 'coupling_operator'

# Use a two-point flux approximation on all grids.
# Note the keyword here: It must be the same as used when assigning the 
# parameters.
tpfa = pp.Tpfa('flow_param')

# Between the grids we use a Robin type coupling (resistance to flow over a fracture).
# Again, the keyword must be the same as used to assign data to the edge
# The edge discretization also needs access to the corresponding discretizations 
# on the neighboring nodes
edge_discretization = pp.RobinCoupling('flow_param_edge', tpfa, tpfa)

# Loop over the nodes in the GridBucket, define primary variables and discretization schemes
for g, d in gb:
    # Assign primary variables on this grid. It has one degree of freedom per cell.
    d[pp.PRIMARY_VARIABLES] = {grid_variable: {"cells": 1, "faces": 0}}
    # Assign discretization operator for the variable.
    # If the discretization is composed of several terms, they can be assigned
    # by multiple entries in the inner dictionary, e.g.
    #  {operator_keyword_1: method_1, operator_keyword_2: method_2, ...}
    d[pp.DISCRETIZATION] = {grid_variable: {operator_keyword: tpfa}}
    
# Loop over the edges in the GridBucket, define primary variables and discretizations
for e, d in gb.edges():
    # 
    g1, g2 = gb.nodes_of_edge(e)
    # The mortar variable has one degree of freedom per cell in the mortar grid
    d[pp.PRIMARY_VARIABLES] = {mortar_variable: {"cells": 1}}
    
    # The coupling discretization links an edge discretization with variables
    # and discretization operators on each neighboring grid
    d[pp.COUPLING_DISCRETIZATION] = {
        coupling_operator_keyword: {
            g1: (grid_variable, operator_keyword),
            g2: (grid_variable, operator_keyword),
            e: (mortar_variable, edge_discretization),
        }
    }
    d[pp.DISCRETIZATION_MATRICES] = {'flow_param_edge': {}}
    
# The task of assembling the linear system is left to a dedicated object:
assembler = pp.Assembler()

# Discretization and assembly of the global linear system can in this case
# be carried out by a single function call.
# Note that for some problems, notably poro-elasticity, this is not possible,
# then discretization must be carried out first.

# Assemble the linear system, using the information stored in the GridBucket
A, b, block_dof, full_dof = assembler.assemble_matrix_rhs(gb)

# Here, A is the global linear system, and b is the corresponding right hand side.
# We can thus obtain the pressure solution by
pressure = sps.linalg.spsolve(A, b)
# The parameters assigned above will not yield a well-posed problem, 
# thus the solve will likely produce a warning about the matrix being singular.
# This can be ignored.

# The ordering of the unknowns in the global linear system will vary depending 
# on how the components in the GridBucket are traversed.
# Untangling the ordering is a two-stage process:
# 1. The system is ordered as a block system, with one block per combination of 
# primary variable and grid or edge (between grids). This information is stored
# in the field block_dof.
# 2. For each block, the local degrees of freedom can be obtained from the field
# full_dof

# To get the block number of a specific primary variable, we need the identifier
# of the relevant component in the GridBucket (either the grid, or the edge between
# grids), and the variable name.
# Getting the grids is easy, there is one in each dimension
g_2d = gb.grids_of_dimension(2)[0]
g_1d = gb.grids_of_dimension(1)[0]

# Formally loop over the edges, there is a single one
for e, _ in gb.edges():
    continue

# Now, the block ordering is obtained, for the 2d grid as 
block_2d = block_dof[(g_2d, grid_variable)]

# full_dof contains the number of dofs per block. To get a global ordering, use
global_dof = np.r_[0, np.cumsum(full_dof)]

# Get 2d dofs
global_dof_2d = np.arange(global_dof[block_2d], global_dof[block_2d+1])
# Print the relevant part of the system matrix
print(A.toarray()[global_dof_2d, :][:, global_dof_2d])